# Read and Visualize a WSI

<a href="https://colab.research.google.com/github/TIA-Lab/tiatoolbox/blob/master/examples/01_example_wsiread.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/TIA-Lab/tiatoolbox/blob/master/examples/01_example_wsiread.ipynb" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Colab"/></a>

_In order to run this notebook on a Kaggle platform, 1) click on the blue Kaggle badge saying_ Open in Kaggle _2) click on_ Settings _on the right of the Kaggle screen, 3) log in to your Kaggle account.

## About this notebook
This jupyter notebook can be run on any computer with a standard browser and no programming language is required. It can run remotely over the Internet, free of charge, thanks to Google Colaboratory or Kaggle. To connect with Colab or Kaggle, click on one of the two blue checkboxes above. Check that "colab" appears in the browser address bar after you click on "Open in Colab". Familiarize yourself with the drop-down menus near the top of the window. You can edit the notebook during the session, for example substituting your own image files for the image files used in this demo. Experiment by changing the parameters of functions. It is not possible for an ordinary user to permanently change this version of the notebook on either Github or colab, so you cannot inadvertently mess it up. Use the notebook's File Menu if you wish to save your own (changed) notebook.

Before running the notebook outside Colab, set up your Python environment, as explained in the 
[README](https://github.com/TIA-Lab/tiatoolbox/blob/master/README.md#install-python-package) file.
### Welcome to tiatoolbox!
This demo reads a whole slide image (WSI) using tiatoolbox. We load a sample WSI, gather some key information, and then extract some image patches. We demo our modules
`wsireader`[details](https://github.com/TIA-Lab/tiatoolbox/blob/master/tiatoolbox/wsicore/wsireader.py) and
`slide_info`[details](https://github.com/TIA-Lab/tiatoolbox/blob/master/tiatoolbox/wsicore/slide_info.py).

### First cell in bash
The first line of code says that the rest of the cell will be interpreted in bash. The second line removes the directory`tmp`if it exists—a previous run may have created it. The other lines set up a suitable Python environment under Colab. No output is expected if the notebook is either run outside Colab or run under Colab for a second time in the same Colab session.   

In [ ]:
%%bash
[ -d tmp ] && ( echo "deleting tmp directory"; rm -rf tmp )
if [ $(env | grep COLAB | wc -c) -gt 0 ] && [ $( pip list | grep datascience | wc -c) -gt 0 ]
then
    echo "uninstalling colab packages with conflicts"
    pip uninstall -y datascience
    pip uninstall -y albumentations
    apt-get -y install libopenjp2-7-dev libopenjp2-tools openslide-tools
    pip install tiatoolbox
fi

We import some some standard Python modules, and also the Python module`wsireader`(see [details](https://github.com/TIA-Lab/tiatoolbox/blob/master/tiatoolbox/wsicore/wsireader.py)) written by the TIA Centre team.

In [ ]:
from tiatoolbox.wsicore import wsireader
import requests
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['figure.dpi'] = 150 # for high resolution figure in notebook

## Reading in a WSI

We load a small WSI, specified with the help of the path variable`user_sample_wsi_path`(default value`None`). Unless this None value is changed, the WSI is downloaded from the web, as seen in the code below, and saved with filename given by the variable`sample_file_name`in the directory given by`data_dir`. Data generated by the notebook is stored under data_dir, providing rapid local access.

In [ ]:
data_dir = './tmp'
sample_file_name = 'sample_wsi_small.svs'

user_sample_wsi_path = None

if user_sample_wsi_path is None:
    sample_wsi_path = '%s/%s' % (data_dir, sample_file_name)
else:
    sample_wsi_path = user_sample_wsi_path
if not os.path.exists(sample_wsi_path):
    os.mkdir(data_dir)
    r = requests.get(
        " https://tiatoolbox.dcs.warwick.ac.uk/sample_wsis/CMU-1-Small-Region.svs"
    )
    with open(sample_wsi_path, "wb") as f:
        f.write(r.content)

Our code shields the user from the incompatible formats produced by different models of scanners from different vendors. The function`wsireader.get_wsireader`has as input a particular WSI, with a particular image format, and outputs an object`wsi_reader_v1`, whose base class is`WSIreader`, and whose derived class depends on the image format. [See details](https://github.com/TIA-Lab/tiatoolbox/blob/master/tiatoolbox/wsicore/wsireader.py). The reader`wsi_reader_v1`provides important information about the WSI. Member functions obtain pixel- or patch-level information, using format-independent code, as we illustrate below.

In [ ]:
wsi_reader_v1 = wsireader.get_wsireader(
                input_img=sample_wsi_path)
print(type(wsi_reader_v1))

First, let's check the basic WSI information, such as magnification, dimension, etc.
(`mpp`= microns per pixel).

In [ ]:
wsi_info = wsi_reader_v1.info.as_dict()
# Print one item per line
print(*list(wsi_info.items()), sep='\n')

### Thumbnail
To see a thumbnail of the WSI, we use the`slide_thumbnail`method of`wsi_reader_v1`.
We load the thumbnail at $\times1.25$ objective power as follows:

In [ ]:
wsi_thumb = wsi_reader_v1.slide_thumbnail(resolution=1.25, units='power')
plt.imshow(wsi_thumb)
plt.axis('off')
plt.show()


## A Mask

Over the next several cells, we will use the WSI object`wsi_thumb`to help 
create a collection of patches covering the tissue area, with limited overlapping. We want to be able to visualise these patches at the high resolution of the original WSI, but it is computationally more efficient to work as much as possible with the much smaller low resolution thumbnail.

The first task is distinguish between tissue and glass (no tissue) in the image. We compute a ***mask***, by which we mean a binary colouring of the pixels to either black=glass or white=tissue. The white area is deliberately made a little larger than than tissue area, as this will be appropriate for our task. Our function`simple_get_mask`binarises the thumbnail on the basis of pixel intensity, using the Otsu method. Morphological operations improve the result. 

In [ ]:
import math, cv2
from skimage import morphology

# an example function using intensity thresholding and morphological operations to obtain tissue regions
def simple_get_mask(rgb):
    gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
    _, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU) # threshold using the OTSU method
    mask = morphology.remove_small_objects(mask == 0, min_size=100, connectivity=2)
    mask = morphology.remove_small_holes(mask, area_threshold=100)
    mask = morphology.binary_dilation(mask, morphology.disk(5))
    return mask

# plot thumbnail (left) with its tissue mask (right)
wsi_thumb_mask = simple_get_mask(wsi_thumb)
plt.subplot(1,2,1)
plt.imshow(wsi_thumb)
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(wsi_thumb_mask, cmap='gray')
plt.axis('off')
plt.show()

### Extracting patches 1: Superpixels and`SLIC`

We wish to extract patches from our WSI. We will construct a set of patches covering the tissue region, without too much overlap. This will be achieved using the *SLIC* algorithm, described [here](https://medium.com/@darshita1405/superpixels-and-slic-6b2d8a6e4f08#:~:text=SLIC%20(Simple%20Linear%20Iterative%20Clustering,proximity%20in%20the%20image%20plane.). Our implementation is`slic`from`skimage`. This program segments the tissue region into disjoint ***superpixels***. A superpixel is, by definition, an irregularly shaped, connected union of pixels with common traits, the traits being chosen according to the problem. In`SLIC`, each superpixel consists of pixels that are near each other in the 2d plane of the image, and are also near (or as near as possible, given the various constraints) to each other in 3d colour space. Pixels cluster according to these traits, while, simultaneously, the program aims for a certain number of superpixels (specified by`nr_expected_superpixels`, one of the arguments to`slic`). The result is a collection of superpixels, each superpixel having approximately the same area. The`numpy`array`wsi_superpixels_mask`, output by`slic`, has the same shape as`wsi_thumb_mask`. The $N$ superpixels are numbered $1,2...N-1,N$. Each low resolution pixel in the $i$-th superpixel corresponds to an entry $i$ in the array, and each pixel that is not in any superpixel corresponds to a zero entry.

In [ ]:
from scipy.ndimage.measurements import center_of_mass
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries

lores_mag = 1.25 # thumbnail objective power (lores = low resolution)
hires_mag = 20 # original WSI objective power (hires = high resolution)
hires_patch_size = 128 # 128x128 original WSI pixels
# map patch size at hires to lores
lores_patch_size = int(hires_patch_size / (hires_mag / lores_mag))
nr_expected_superpixels = math.ceil(np.sum(wsi_thumb_mask) / (lores_patch_size ** 2))
    
wsi_superpixels_mask = slic(wsi_thumb,
                    mask=wsi_thumb_mask,
                    n_segments=nr_expected_superpixels,
                    compactness=1000,
                    sigma=1)

print('#Actual Patches / #Expected Patches : %d/%d' % (np.unique(wsi_superpixels_mask).shape[0], nr_expected_superpixels))

 
In the image below, the position of each superpixel is made clear by marking the boundary in yellow. Each centre (blue dot in the image below) of mass of each superpixel is used as the centre of a new square patch. The algorithm encourages the centres to arrange themselves, where possible, in the pattern of a square grid, usually tilted. The patches are all of the same size, and they cover the tissue area. They may overlap, but the overlap is limited. To locate these centres, we need a coordinate system. The algorithm encourages the centres to arrange themselves roughly in the pattern of a square grid, usually tilted. Instead of the familiar`(x,y)`coordinates, we use`(row,column)`coordinates because our data is held in arrays. The origin is at the top-left of the image, and the first coordinate increases as one moves down. Up to this point, we have worked at low resolution for computational efficiency. But examination of each patch in detail requires the use of high resolution data, and we change coordinates accordingly.

In [ ]:
lores_superpixels_center = center_of_mass(wsi_superpixels_mask,
                labels=wsi_superpixels_mask,
                index=np.unique(wsi_superpixels_mask)[1:]) # omit zeros–they correspond to pixels not in any superpx
lores_superpixels_center = np.array(lores_superpixels_center) # coordinates Y,X because 2d-array uses row,col
lores_superpixels_center = lores_superpixels_center.astype(np.int32)
selected_indices = wsi_thumb_mask[lores_superpixels_center[:,0],lores_superpixels_center[:,1]]
lores_superpixels_center = lores_superpixels_center[selected_indices]

# show the patches region and their centres of mass
plt.imshow(mark_boundaries(wsi_thumb, wsi_superpixels_mask))
plt.scatter(lores_superpixels_center[:,1], lores_superpixels_center[:,0], s=2)
plt.axis('off')
plt.show()

We then convert the centres of each region to the top-left position of the patches at high resolution.

In [ ]:
# convert to top left idx at hires_mag level
lores_superpixels_top_left = (lores_superpixels_center - (lores_patch_size // 2))
hires_superpixels_top_left = lores_superpixels_top_left * (hires_mag / lores_mag)
hires_superpixels_top_left = hires_superpixels_top_left.astype(np.int32)

We will now load some patches for visualisation.

In [ ]:
nr_viz_patches = 16

# for illustration purpose, we only visualise a small number of examples
selected_indices = np.random.randint(0, hires_superpixels_top_left.shape[0], size=(4*nr_viz_patches,))
hires_superpixel_top_left = hires_superpixels_top_left[selected_indices]

patch_list = []
for patch_coord in hires_superpixels_top_left:
    patch = wsi_reader_v1.read_region(
                location=patch_coord[::-1],
                level=0, size=hires_patch_size)
    patch_list.append(patch)

# plot the first 16
sub_patches = np.array(patch_list[:16])
sub_patches = np.reshape(sub_patches, (4, 4, hires_patch_size, hires_patch_size, 3))
sub_patches = np.transpose(sub_patches, (0, 2, 1, 3, 4))
sub_patches = np.reshape(sub_patches, (4 * hires_patch_size, 4 * hires_patch_size, 3))
plt.imshow(sub_patches)
plt.axis('off')
plt.show()


If you want to extract the entire WSI (including the background), you can use the built-in`save_tiles`functionality of the`WSIReader`object.

We start by creating another`WSIReader`object and then save tiles using the`save_tiles`function with the keywords`tile_objective_value`,`tile_read_size`and`output_dir`. These correspond to the magnification set for reading patches, their expected width & height and the output destination. The word **tile** refers to a very large image patch. Here, tiles are read at 20x objective magnification, each of size $1000\times1000$, and will be saved in the`data_dir`.

In [ ]:
# create a file handler
wsi_reader_v2 = wsireader.get_wsireader(
                input_img=sample_wsi_path
                )
wsi_reader_v2.save_tiles(
                output_dir=data_dir + '/output/',
                tile_objective_value=20,
                tile_read_size=(1000, 1000)
                )


We check the content of the output folder and visualize some tiles. The extracted tiles are saved in the directory`{data_dir}/output/{sample_file_name}`. The folder contains`Output.csv`summarizing the extracted tiles.

In [ ]:
import pandas as pd

tile_summary = pd.read_csv('{data_dir}/output/{sample_file_name}/Output.csv'.format(
                                data_dir=data_dir, sample_file_name=sample_file_name))
print(tile_summary)


 We plot`Tile_20_1000_1000.jpg`as an example.

In [ ]:
sample_tile = cv2.imread('%s/output/%s/%s' % (data_dir, sample_file_name, tile_summary.iloc[4]['Tile_Name']))
sample_tile = cv2.cvtColor(sample_tile, cv2.COLOR_BGR2RGB)

plt.imshow(sample_tile)
plt.axis('off')
plt.show()


## jp2 format

The base class`WSIReader`also contains a derived class for WSIs in`jp2`format (see above for more details).We will briefly illustrate this by downloading a`jp2`WSI from the internet and visualize its thumbnail.

In [ ]:

sample_wsi_path = '%s/sample_jp2.jp2' % data_dir
if not os.path.exists(sample_wsi_path):
    r = requests.get(
        "https://tiatoolbox.dcs.warwick.ac.uk/sample_wsis/test1.jp2"
    )
    with open(sample_wsi_path, "wb") as f:
        f.write(r.content)

wsi_reader_v3 = wsireader.get_wsireader(
                input_img=sample_wsi_path)
print(type(wsi_reader_v3))
wsi_thumb = wsi_reader_v3.slide_thumbnail(resolution=1.25, units='power')
plt.imshow(wsi_thumb)
plt.axis('off')
plt.show()